In [11]:
import tensorflow as tf
from tensorflow.keras import datasets,layers, models
from tensorflow import keras
import math
import numpy as np
from skimage import color
from matplotlib import pyplot as plt
(imgs,lbls), (tst_imgs,tst_lbls) = datasets.cifar10.load_data()

imgs, tst_imgs = imgs / 255.0, tst_imgs / 255.0

In [3]:
inps = keras.Input(shape=imgs.shape[1:])

interl = lambda: layers.Dense(20, activation="relu")

summary = layers.Dense(10, activation="softmax")
def pmid(inp,dropout=None):
    sqsz = 7
    
    convs = [layers.MaxPooling2D((2,2),(2,2))(layers.Conv2D(2**i, (sqsz-i,sqsz-i), activation="relu")(inp)) for i in range(1,sqsz-1)]
    inter = [layers.Flatten()(interl()(c)) for c in convs]
    
    if dropout is not None:
        return layers.Dropout(dropout)(layers.Concatenate()(inter)), convs
    return layers.Concatenate()(inter), convs


inter1, pmid1 = pmid(inps)
secondarys = [pmid(pm) for pm in pmid1]
inter2 = layers.Concatenate()([s[0] for s in secondarys])
inter4, pmid4 = pmid(inps,dropout=0.4)
secondarys4 = [pmid(pm,dropout=0.4) for pm in pmid1]
inter5 = layers.BatchNormalization()(layers.Concatenate()([s[0] for s in secondarys4]))
model = keras.Model(inps, summary(inter2), name="cifar10cnn")
model_dropout = keras.Model(inps, summary(inter5))
model.summary()
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model_dropout.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

Model: "cifar10cnn"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_60 (Conv2D)             (None, 27, 27, 2)    218         ['input_2[0][0]']                
                                                                                                  
 conv2d_61 (Conv2D)             (None, 28, 28, 4)    304         ['input_2[0][0]']                
                                                                                                  
 conv2d_62 (Conv2D)             (None, 29, 29, 8)    392         ['input_2[0][0]']                
                                                                                         

 max_pooling2d_66 (MaxPooling2D  (None, 4, 4, 4)     0           ['conv2d_66[0][0]']              
 )                                                                                                
                                                                                                  
 max_pooling2d_67 (MaxPooling2D  (None, 5, 5, 8)     0           ['conv2d_67[0][0]']              
 )                                                                                                
                                                                                                  
 max_pooling2d_68 (MaxPooling2D  (None, 5, 5, 16)    0           ['conv2d_68[0][0]']              
 )                                                                                                
                                                                                                  
 max_pooling2d_69 (MaxPooling2D  (None, 6, 6, 32)    0           ['conv2d_69[0][0]']              
 )        

                                                                                                  
 dense_73 (Dense)               (None, 5, 5, 20)     100         ['max_pooling2d_71[0][0]']       
                                                                                                  
 dense_74 (Dense)               (None, 5, 5, 20)     180         ['max_pooling2d_72[0][0]']       
                                                                                                  
 dense_75 (Dense)               (None, 6, 6, 20)     340         ['max_pooling2d_73[0][0]']       
                                                                                                  
 dense_76 (Dense)               (None, 6, 6, 20)     660         ['max_pooling2d_74[0][0]']       
                                                                                                  
 dense_77 (Dense)               (None, 4, 4, 20)     60          ['max_pooling2d_75[0][0]']       
          

 flatten_87 (Flatten)           (None, 720)          0           ['dense_89[0][0]']               
                                                                                                  
 flatten_88 (Flatten)           (None, 720)          0           ['dense_90[0][0]']               
                                                                                                  
 flatten_89 (Flatten)           (None, 980)          0           ['dense_91[0][0]']               
                                                                                                  
 concatenate_15 (Concatenate)   (None, 2360)         0           ['flatten_65[0][0]',             
                                                                  'flatten_66[0][0]',             
                                                                  'flatten_67[0][0]',             
                                                                  'flatten_68[0][0]',             
          

In [ ]:
model_dropout.summary()

In [4]:
CNN_trained = model.fit(imgs,lbls,epochs=10,validation_data=(tst_imgs,tst_lbls))

Epoch 1/10


2022-11-11 23:17:39.661505: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-11 23:17:41.274572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - ETA: 0s - loss: 1.2998 - accuracy: 0.5419

2022-11-11 23:18:52.678635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 80s 49ms/step - loss: 1.2998 - accuracy: 0.5419 - val_loss: 1.0656 - val_accuracy: 0.6268
Epoch 2/10
1563/1563 [==============================] - 74s 47ms/step - loss: 0.9693 - accuracy: 0.6657 - val_loss: 0.9331 - val_accuracy: 0.6733
Epoch 3/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.8447 - accuracy: 0.7087 - val_loss: 0.9307 - val_accuracy: 0.6790
Epoch 4/10
1563/1563 [==============================] - 73s 47ms/step - loss: 0.7535 - accuracy: 0.7424 - val_loss: 0.8687 - val_accuracy: 0.7053
Epoch 5/10
1563/1563 [==============================] - 73s 47ms/step - loss: 0.6766 - accuracy: 0.7673 - val_loss: 0.8753 - val_accuracy: 0.7025
Epoch 6/10
1563/1563 [==============================] - 73s 47ms/step - loss: 0.6025 - accuracy: 0.7920 - val_loss: 0.9080 - val_accuracy: 0.6998
Epoch 7/10
1563/1563 [==============================] - 73s 47ms/step - loss: 0.5352 - accuracy: 0.8148 - val_loss: 0.9143 - val_accura

In [5]:
print(CNN_trained.history['accuracy'])

[0.5419000387191772, 0.6656800508499146, 0.7086600661277771, 0.742400050163269, 0.7672800421714783, 0.7919800281524658, 0.8147600293159485, 0.8342600464820862, 0.854360044002533, 0.8700800538063049]


In [6]:
(loss, acc) = model.evaluate(tst_imgs, tst_lbls)
print(acc)

313/313 [==============================] - 6s 20ms/step - loss: 1.0282 - accuracy: 0.7045
0.7045000195503235


In [7]:
predictions = model.predict(tst_imgs)

2022-11-11 23:30:07.597801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 7s 20ms/step


In [8]:
wrongs = [(np.round(pred), tst_imgs[ix], tst_lbls[ix],ix) for ix, pred in enumerate(predictions)]

In [9]:
def augment(img):
    def flipped(i):
        return [np.rot90(i, k=j) for j in range(4)]
    return flipped(color.rgb2gray(img))

In [ ]:
new_imgs, new_lbls = [], []
for ix,img in enumerate(imgs):
    this_augs = augment(img)
    new_imgs.append(this_augs[-1])
    new_imgs.append(img)
    new_lbls.append(lbls[ix])
    new_lbls.append(lbls[ix])

import random    
plt.figure(figsize=(4,4))
i=1
for img in random.sample(list(imgs), k=2):
    plt.subplot(4,4,i)
    i+=1
    plt.grid(False)
    plt.imshow(img)
    for a in augment(img):
        plt.subplot(4,4,i)
        i+=1
        plt.grid(False)
        plt.imshow(a)

In [ ]:
print('hi')
CNN_trained2 = model.fit(new_imgs,new_lbls,epochs=10,validation_data=(tst_imgs,tst_lbls))

CNN_trained3 = model_dropout.fit(new_imgs,new_lbls,epochs=10,validation_data=(tst_imgs,tst_lbls))

The network did not end up performing any better on the validation set after doing a dropout. Kinda confusing to me. But my accuracy was never very good, at 67%.